## TODO: create sidekick agent with playwright

##### NOTE: make sure you install playwright package

In [2]:
from typing import Annotated
from typing_extensions import TypedDict
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode, tools_condition
from langchain_openai import ChatOpenAI
from langgraph.checkpoint.memory import MemorySaver

from langchain.tools import tool
from langchain_community.agent_toolkits import PlayWrightBrowserToolkit
from langchain_community.tools.playwright.utils import create_async_playwright_browser

from IPython.display import Image, display
import gradio as gr
import requests
import os

from dotenv import load_dotenv
load_dotenv(override=True)

/home/anh/workspace/course-udemy-agentic-enginer/.venv./lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


True

#### We will use asynchronous langgraph

##### Run a tool in async mode
```
#tool.run(inputs)
await tool.arun(inputs)
```

##### Invoke a graph in async mode
```
#graph.invoke(state)
await graph.ainvoke(state)
```

#### Tools

In [3]:
from langchain_community.utilities import GoogleSerperAPIWrapper #what is this?
serper = GoogleSerperAPIWrapper()
#serper.run("What is the capital of France") #example running

pushover_token=os.getenv("PUSHOVER_API_KEY")
pushover_user=os.getenv("PUSHOVER_USER_KEY")
pushover_url="https://api.pushover.net/l/messages.json"

@tool("search")
def tool_search(query: str) -> str:
    """Use to externally search information online."""
    print (f"Performing web search...")
    return serper.run(query)

@tool("push")
def tool_push(msg: str):
    """Use to push a notification, or message, or text to the user's device"""
    requests.post(pushover_url, data={"token": pushover_token, "user": pushover_user, "message": msg})
    print (f"Message {msg[:5]} is pushed")
    return 

#### Declare nest_api for temporarily use

In [4]:
import nest_asyncio
nest_asyncio.apply()

In [5]:
!export PATH="/home/anh/.nvm/versions/node/v24.11.1/bin/node:$PATH" 



In [6]:
print (os.environ["PATH"])
os.environ["PATH"] = "/home/anh/.nvm/versions/node/v24.11.1/bin/node:" + os.environ["PATH"]

/home/anh/workspace/course-udemy-agentic-enginer/.venv./bin:/home/anh/.local/bin:/home/anh/.local/bin:/home/anh/workspace/local_utils:/home/anh/.local/bin:/home/anh/miniforge3/bin:/home/anh/miniforge3/condabin:/home/anh/.local/bin:/home/anh/.local/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/usr/games:/usr/local/games:/snap/bin:/snap/bin


#### Use langchain playwright toolkit``

In [7]:


#NOTE: use the below command to update related packages to prevent error
# pip install playwright
# uv add --upgrade langchain-community langchain pydantic


async_browser = create_async_playwright_browser(headless=False)
toolkit = PlayWrightBrowserToolkit.from_browser(async_browser=async_browser)

tools = toolkit.get_tools()

all_tools = tools + [tool_push]

In [8]:
for tool in tools:
    print (f"tool: {tool.name}={tool}")

tool: click_element=async_browser=<Browser type=<BrowserType name=chromium executable_path=/home/anh/.cache/ms-playwright/chromium-1200/chrome-linux64/chrome> version=143.0.7499.4>
tool: navigate_browser=async_browser=<Browser type=<BrowserType name=chromium executable_path=/home/anh/.cache/ms-playwright/chromium-1200/chrome-linux64/chrome> version=143.0.7499.4>
tool: previous_webpage=async_browser=<Browser type=<BrowserType name=chromium executable_path=/home/anh/.cache/ms-playwright/chromium-1200/chrome-linux64/chrome> version=143.0.7499.4>
tool: extract_text=async_browser=<Browser type=<BrowserType name=chromium executable_path=/home/anh/.cache/ms-playwright/chromium-1200/chrome-linux64/chrome> version=143.0.7499.4>
tool: extract_hyperlinks=async_browser=<Browser type=<BrowserType name=chromium executable_path=/home/anh/.cache/ms-playwright/chromium-1200/chrome-linux64/chrome> version=143.0.7499.4>
tool: get_elements=async_browser=<Browser type=<BrowserType name=chromium executable_

In [11]:
tool_dict = {tool.name: tool for tool in tools}

tool_navigate = tool_dict.get("navigate_browser")
tool_extract_text =tool_dict.get("extract_text")


# test run
# await tool_navigate.arun({"url": "https://www.cnn.com"})
# text = await tool_extract_text.arun({})
# import textwrap
# print (textwrap.fill(text[:200]))

In [12]:
##### Print out the text content of the web browswer above

### define an LLM-based chatbot with these tools

In [13]:
class State(TypedDict):
    messages: Annotated[list, add_messages]

llm_gpt4omini = ChatOpenAI(model="gpt-4o-mini")
llm_ollama = ChatOpenAI(
    model="qwen3-30b-instruct:latest",               # whatever model name you use in `ollama run`
    base_url="http://localhost:11434/v1",
    api_key="ollama-not-used",        # Ollama ignores it but ChatOpenAI requires *something*
    temperature=0.5,
)

llm = llm_ollama
llm_with_tools = llm.bind_tools(all_tools)

def chatbot(state: State):
    
    response = llm_with_tools.invoke(state['messages'])
    return {"messages": [response]}

ChatOpenAI(profile={}, client=<openai.resources.chat.completions.completions.Completions object at 0x79a2a12a0140>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x79a2a0e2c8f0>, root_client=<openai.OpenAI object at 0x79a2a12a07a0>, root_async_client=<openai.AsyncOpenAI object at 0x79a2a12a01a0>, model_name='qwen3-30b-instruct:latest', temperature=0.5, model_kwargs={}, openai_api_key=SecretStr('**********'), openai_api_base='http://localhost:11434/v1')

### Degine the graph with tool node

In [15]:
#def tools_condition(state: State): return too
graph_builder = StateGraph(State)
graph_builder.add_node("node_chatbot", chatbot)
graph_builder.add_node("node_tools", ToolNode(tools=all_tools))
graph_builder.add_conditional_edges("node_chatbot", tools_condition, {"tools": "node_tools", "__end__": END}) # __end__ matches tools_condition output
graph_builder.add_edge("node_tools", "node_chatbot")
graph_builder.add_edge(START, "node_chatbot")

memory_ram = MemorySaver()
graph = graph_builder.compile(checkpointer=memory_ram)

In [16]:
config = {"configurable": {"thread_id": "10"}}
async def chat(user_input: str, history):
    # we do not use chat history, but directly use memory langchain
    result = await graph.ainvoke({"messages": [{"role": "user", "content": user_input}]}, config=config)
    return result["messages"][-1].content

ui = gr.ChatInterface(fn=chat)
ui.queue()        # enables async
ui.launch()

* Running on local URL:  http://127.0.0.1:7861
* To create a public link, set `share=True` in `launch()`.


Traceback (most recent call last):
  File "/home/anh/workspace/course-udemy-agentic-enginer/.venv./lib/python3.12/site-packages/gradio/queueing.py", line 763, in process_events
    response = await route_utils.call_process_api(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/anh/workspace/course-udemy-agentic-enginer/.venv./lib/python3.12/site-packages/gradio/route_utils.py", line 354, in call_process_api
    output = await app.get_blocks().process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/anh/workspace/course-udemy-agentic-enginer/.venv./lib/python3.12/site-packages/gradio/blocks.py", line 2106, in process_api
    result = await self.call_function(
             ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/anh/workspace/course-udemy-agentic-enginer/.venv./lib/python3.12/site-packages/gradio/blocks.py", line 1586, in call_function
    prediction = await fn(*processed_input)
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/anh/workspace/cour

In [34]:
state_info = graph.get_state(config).values



In [40]:
for msg in state_info['messages']:
    print (msg)


content='hello?' additional_kwargs={} response_metadata={} id='998a539a-4ec0-494e-9c3e-cdb5f91414b3'
content='Hello! How can I assist you today?' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 271, 'total_tokens': 281, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_aa07c96156', 'id': 'chatcmpl-CkyHeZBDjPgmEA15m9seoW1RJIZQ0', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None} id='lc_run--019b04b9-f5b7-78a2-8e36-aca9c0325ffd-0' usage_metadata={'input_tokens': 271, 'output_tokens': 10, 'total_tokens': 281, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}
content='what is my name?' additional_kwargs=

Message CNN H is pushed
Message CNBC  is pushed


Traceback (most recent call last):
  File "/home/anh/workspace/course-udemy-agentic-enginer/.venv./lib/python3.12/site-packages/gradio/queueing.py", line 763, in process_events
    response = await route_utils.call_process_api(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/anh/workspace/course-udemy-agentic-enginer/.venv./lib/python3.12/site-packages/gradio/route_utils.py", line 354, in call_process_api
    output = await app.get_blocks().process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/anh/workspace/course-udemy-agentic-enginer/.venv./lib/python3.12/site-packages/gradio/blocks.py", line 2106, in process_api
    result = await self.call_function(
             ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/anh/workspace/course-udemy-agentic-enginer/.venv./lib/python3.12/site-packages/gradio/blocks.py", line 1586, in call_function
    prediction = await fn(*processed_input)
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/anh/workspace/cour

In [38]:
 graph.get_state(config)

StateSnapshot(values={'messages': [HumanMessage(content='hello?', additional_kwargs={}, response_metadata={}, id='998a539a-4ec0-494e-9c3e-cdb5f91414b3'), AIMessage(content='Hello! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 271, 'total_tokens': 281, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_aa07c96156', 'id': 'chatcmpl-CkyHeZBDjPgmEA15m9seoW1RJIZQ0', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--019b04b9-f5b7-78a2-8e36-aca9c0325ffd-0', usage_metadata={'input_tokens': 271, 'output_tokens': 10, 'total_tokens': 281, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio'